In [1]:
import numpy as np
import pandas as pd
import torch
import functools
import operator

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
df_train = pd.read_pickle("../data/df_train.pkl")
df_test = pd.read_pickle("../data/df_test.pkl")

In [4]:
df_train_X = df_train.drop(columns = ["cleavage_freq"])
y = df_train["cleavage_freq"]

### Preprocessing

In [5]:
class Preprocessing:
    def fold_seq(df):
        df["stacked"] = df["grna_target_sequence"].apply(lambda x: x.tolist()) + df["target_sequence"].apply(lambda x: x.tolist())
        df["stacked"] = df["stacked"].apply(lambda x: np.array(x))
    
    def tensorfy(stacked):
        temp = []
        for i in stacked:
            temp.append(i)
        return torch.from_numpy(np.array(temp))

In [6]:
Preprocessing.fold_seq(df_train_X)
X = Preprocessing.tensorfy(df_train_X["stacked"])
X

tensor([[0.7500, 0.7500, 0.5000,  ..., 0.0000, 0.0000, 0.0000],
        [0.7500, 0.2500, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.7500, 1.0000, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.7500, 0.5000, 0.7500,  ..., 0.0000, 0.0000, 0.0000],
        [0.7500, 0.2500, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.5000, 0.7500, 0.2500,  ..., 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

### Model